In [1]:
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]

# Preprocessing:
Preprocess the query and document. Simplest version of preprocessing.
Make lowercase
Split by spaces

Jaccard similarity:
Takes the intersection and the union of set

In [2]:
def jaccard_similarity(query, document):
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)


Function to return the best document to user based on similarity between query and corpus

In [3]:
def return_response(query, corpus):
    similarities = []
    for doc in corpus:
        similarity = jaccard_similarity(query, doc)
        similarities.append(similarity)
    return corpus_of_documents[similarities.index(max(similarities))]

In [4]:
user_prompt = "What is a leisure activity that you like?"
user_input = "I like to hike"

In [5]:
return_response(user_input, corpus_of_documents)

'Go for a hike and admire the natural scenery.'

# Adding in a LLM
Adding in ollama

In [7]:
import requests
import json

/Users/jonathantsegaye/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


First we define the inputs. To work with ollama we're going to take

1. user input
2. fetch the most similar document (as measured by our similarity measure)
3. pass that into a prompt to the language model
4. then return the result to the user

The new term is *prompt*. It's the instructions that you provide to the LLM.

In [8]:
user_input = "I like to hike"
relevant_document = return_response(user_input, corpus_of_documents)
full_response = []

prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""

Now we make the API call to ollama. Make sure ollama is running already on your local machine by running `ollama serve`

In [9]:
url = 'http://localhost:11434/api/generate'

data = {
    "model" : "llama2",
    "prompt" : prompt.format(user_input=user_input, relevant_document=relevant_document)
}

headers = {'Content-Type': 'application/json'}

response = requests.post(url, data=json.dumps(data), stream=True)

try:
    count = 0
    for line in response.iter_lines():
        # filter out keep-alive new lines
        # count += 1
        # if count % 5 == 0:
        #   print(decoded_line['response']) # print every fifth token
        if line:
            decoded_line = json.loads(line.decode('utf-8'))

            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

 Great! Based on your input, I recommend going for a hike in a nearby nature reserve. The beautiful scenery and peaceful atmosphere will definitely be worth it. Enjoy your hike!
